# Assignment


In [1]:
import pandas as pd
import geopandas as gpd
import psycopg2
import json
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, String
from shapely.geometry import shape

## Clear data

Collate the data in "SA2.zip",get the SA2code, SA2name data content and GCC-related information.And filter out the line where the blank information is.Finally, we check whether all the geometry objects in the geometry column are valid and not duplicated.

In [2]:
SA2 = gpd.read_file("SA2/SA2_2021_AUST_GDA2020.shp")
columns_to_keep = ['SA2_CODE21', 'SA2_NAME21','GCC_CODE21', 'GCC_NAME21', 'geometry']
SA2 = SA2[columns_to_keep]
# Canonical column name
SA2.rename(columns={'SA2_CODE21': 'sa2_code'}, inplace=True)
SA2.rename(columns={'SA2_NAME21': 'sa2_name'}, inplace=True)
SA2.rename(columns={'GCC_CODE21': 'gcc_code'}, inplace=True)
SA2.rename(columns={'GCC_NAME21': 'gcc_name'}, inplace=True)
SA2.dropna(inplace=True)

SA2.drop_duplicates(subset=['geometry'], inplace=True)
valid_geom = SA2['geometry'].apply(lambda x: shape(x).is_valid)
SA2 = SA2[valid_geom]
SA2

,sa2_code,sa2_name,gcc_code,gcc_name,geometry
0,101021007,Braidwood,1RNSW,Rest of NSW,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,1RNSW,Rest of NSW,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,1RNSW,Rest of NSW,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,1RNSW,Rest of NSW,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,1RNSW,Rest of NSW,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."
...,...,...,...,...,...
2463,801111141,Namadgi,8ACTE,Australian Capital Territory,"POLYGON ((148.80407 -35.37619, 148.80417 -35.3..."
2466,901011001,Christmas Island,9OTER,Other Territories,"POLYGON ((105.67393 -10.41566, 105.67399 -10.4..."
2467,901021002,Cocos (Keeling) Islands,9OTER,Other Territories,"MULTIPOLYGON (((96.91512 -12.14044, 96.91513 -..."
2468,901031003,Jervis Bay,9OTER,Other Territories,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."


Collate the data in "Stops.txt",some content that was not relevant for later analysis was deleted, leaving the content related to the platforms in the database and their latitude and longitude data.

In [3]:
Stops = pd.read_csv("Stops.txt", delimiter=",")
columns_to_keep = ['stop_id','stop_code','stop_name', 'stop_lat','stop_lon']
Stops = Stops[columns_to_keep]
# Canonical column name
Stops.rename(columns={'stop_lat': 'latitude'}, inplace=True)
Stops.rename(columns={'stop_lon': 'longitude'}, inplace=True)

Collate the data in "Businesses.csv", convert the data content to the corresponding data format,and filtered the data, keeping only data on industry type, sa2 and total number of businesses.And check if the value of SA2 is valid.

In [4]:
Businesses =  pd.read_csv('Businesses.csv')
Businesses['total_businesses'] = Businesses['total_businesses'].astype(int)
columns_to_keep = ['industry_code','industry_name','sa2_code','sa2_name','total_businesses']
Businesses = Businesses[columns_to_keep]
compare = SA2['sa2_code'].astype(int)
Businesses['match'] = Businesses['sa2_code'].isin(compare)
Businesses = Businesses[Businesses['match'] == True].drop(columns=['match'])

Collate the data in "PollingPlaces2019.csv",some content that was not relevant for later analysis was removed, and data with blank values and duplicate coordinates are also eliminated.

In [5]:
Polls = pd.read_csv("PollingPlaces2019.csv")
Polls.drop_duplicates(subset=['the_geom'], inplace=True)
columns_to_keep = ['polling_place_id','polling_place_name','latitude','longitude']
Polls = Polls[columns_to_keep]
Polls.dropna(inplace=True)

Collate the data in "Catchments.zip", combine the three data information into a data table, in descending order of USE_ID, because in task2, the 0-19 year old people are merged into "young people ", so the" CATCH_TYPE "," ADD_DATE","KINDERGART","YEAR1-12" columns are all removed. Keep only columns for "USE_ID "," USE_DESC","geometry".And filter out the line where the blank information is. Finally, we check whether all the geometry objects in the geometry column are valid and not duplicated.And convert CRS to 7844.

In [6]:
School_future = gpd.read_file("Catchments/catchments/catchments_future.shp")
School_primary = gpd.read_file("Catchments/catchments/catchments_primary.shp")
School_secondary = gpd.read_file("Catchments/catchments/catchments_secondary.shp")
merged_schools = gpd.GeoDataFrame(pd.concat([School_future, School_primary, School_secondary], ignore_index=True))
columns_to_keep = ['USE_ID','USE_DESC','geometry']
School = merged_schools[columns_to_keep].copy()
School['USE_ID'] = School['USE_ID'].astype(int)
School.dropna(inplace=True)

School.drop_duplicates(subset=['geometry'], inplace=True)
School.drop_duplicates(subset=['USE_ID'], inplace=True)

valid_geom = School['geometry'].apply(lambda x: shape(x).is_valid)
School = School[valid_geom]

School['geometry'] = School['geometry'].to_crs(epsg=7844)

# Canonical column name
School.rename(columns={'USE_ID': 'use_id'}, inplace=True)
School.rename(columns={'USE_DESC': 'use_desc'}, inplace=True)

Collate the data in "Population.csv", convert the data content to the corresponding data format, and add a column named "young_people" to record young people aged 0-19 in the current region.

In [7]:
Population = pd.read_csv("Population.csv")

Population['0-4_people'] = Population['0-4_people'].astype(int)
Population['5-9_people'] = Population['5-9_people'].astype(int)
Population['10-14_people'] = Population['10-14_people'].astype(int)
Population['15-19_people'] = Population['15-19_people'].astype(int)

young_people = Population['0-4_people'] + Population['5-9_people'] + Population['10-14_people'] + Population['15-19_people']
Population['young_people'] = young_people

Population['sa2_code'] = Population['sa2_code'].astype(int)
Population['total_people'] = Population['total_people'].astype(int)
columns_to_keep = ['sa2_code','sa2_name','young_people','total_people']
Population = Population[columns_to_keep]

Collate the data in "incomes.csv", convert the data content into the corresponding data format, clear the abnormal information, and add the column "total_income" to record the total Income in the current region.

In [8]:
Income = pd.read_csv("Income.csv")

contains_np = Income.apply(lambda row: 'np' in row.values, axis=1)
Income = Income[~contains_np]

Income['earners'] = Income['earners'].astype(int)
Income['mean_income'] = Income['mean_income'].astype(int)
total_income = Income['earners'] * Income['mean_income']
Income['total_income'] = total_income
# Canonical column name
Income.rename(columns={'sa2_code21': 'sa2_code'}, inplace=True)


## Import all datasets  PostgreSQL server

Use the pgconnect function to connect to the PostgreSQL server and load the PostGIS extension.

In [9]:
def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

In [10]:
credentials = "Credentials.json"
db, conn = pgconnect(credentials)

Connected successfully.


In [11]:
conn.execute("CREATE EXTENSION IF NOT EXISTS postgis;")

Create the structure of the table, define foreign keys and primary keys to establish associations between different tables.

In [12]:
# SA2
db.execute('''
CREATE TABLE sa2 (
    sa2_code VARCHAR(255) PRIMARY KEY,
    sa2_name VARCHAR(255),
    GCC_code VARCHAR(255),
    GCC_name VARCHAR(255),
    geometry geometry(MULTIPOLYGON, 7844)
)
''')

In [13]:
# Stops
db.execute('''
CREATE TABLE stops (
    stop_id VARCHAR(255) PRIMARY KEY,
    stop_code VARCHAR(255),
    stop_name VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT,
    sa2_code VARCHAR(255),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [14]:
# Businesses
db.execute('''
CREATE TABLE businesses (
    Industry_code VARCHAR(255),
    industry_name VARCHAR(255),
    sa2_code VARCHAR(255),
    sa2_name VARCHAR(255),
    total_businesses INTEGER,
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [15]:
# Polls
db.execute('''
CREATE TABLE polls (
    polling_place_id INTEGER PRIMARY KEY,
    polling_place_name VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT,
    sa2_code VARCHAR(255),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [16]:
# School
db.execute('''
CREATE TABLE school (
    USE_ID INTEGER PRIMARY KEY,
    USE_DESC VARCHAR(255),
    geometry geometry(MULTIPOLYGON, 7844),
    sa2_code VARCHAR(255),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [17]:
# Population
db.execute('''
CREATE TABLE population (
    sa2_code VARCHAR(255),
    sa2_name VARCHAR(255),
    young_people INTEGER,
    total_people INTEGER,
    PRIMARY KEY (sa2_code),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [18]:
# Income
db.execute('''
CREATE TABLE income (
    sa2_code VARCHAR(255),
    sa2_name VARCHAR(255),
    earners INTEGER,
    median_age FLOAT,
    median_income FLOAT,
    mean_income FLOAT,
    total_income FLOAT,
    PRIMARY KEY (sa2_code),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In the Businesses, Population, and Income tables, the SA2_code column is defined as a foreign key, which references the SA2_code column in the SA2 table as the primary key. This establishes an association between these tables.

Load the cleaned data to the server one by one.

In [19]:
SA2.to_postgis('sa2', db, index=False,if_exists='append')

In [20]:
Polls.to_sql('polls', db, index=False, if_exists='append')

543

In [21]:
Population.to_sql('population', db, index=False, if_exists='append')
Income.to_sql('income', db, index=False, if_exists='append')
Stops.to_sql('stops', db, index=False, if_exists='append')
Polls.to_sql('polls', db, index=False, if_exists='append')

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "polls_pkey"
DETAIL:  Key (polling_place_id)=(58) already exists.

[SQL: INSERT INTO polls (polling_place_id, polling_place_name, latitude, longitude) VALUES (%(polling_place_id)s, %(polling_place_name)s, %(latitude)s, %(longitude)s)]
[parameters: ({'polling_place_id': 58, 'polling_place_name': 'Oatley', 'latitude': -33.9847, 'longitude': 151.081}, {'polling_place_id': 392, 'polling_place_name': 'Dharruk', 'latitude': -33.7475, 'longitude': 150.817}, {'polling_place_id': 31, 'polling_place_name': 'Allawah', 'latitude': -33.9767897, 'longitude': 151.1148974}, {'polling_place_id': 67, 'polling_place_name': 'Allawah South', 'latitude': -33.9756, 'longitude': 151.111}, {'polling_place_id': 56500, 'polling_place_name': 'Beverly Hills North (Banks)', 'latitude': -33.9413, 'longitude': 151.075}, {'polling_place_id': 79612, 'polling_place_name': 'Beverly Hills South (Banks)', 'latitude': -33.9528, 

In [22]:
School.to_postgis('school', db, index=False, if_exists='append')

In [23]:
Businesses.to_sql('businesses', db, index=False, if_exists='append')

198

Next, associate SA2_code with SA2 by obtaining the latitude and longitude from Stops, Polls, and School tables.And if sa2 does not match the content, delete the data.

In [24]:
db.execute('''
ALTER TABLE Stops
ADD COLUMN geom GEOMETRY(Point, 4326);
UPDATE Stops
SET geom = ST_SetSRID(ST_MakePoint(longitude, latitude), 4326);
ALTER TABLE Stops
ADD COLUMN geometry_7844 GEOMETRY(Point, 7844);
UPDATE Stops
SET geometry_7844 = ST_Transform(geom, 7844);

UPDATE Stops s
SET sa2_code = sa2.sa2_code
FROM SA2 sa2
WHERE ST_Within(s.geometry_7844, sa2.geometry);

DELETE FROM Stops
WHERE sa2_code IS NULL;

ALTER TABLE Stops
DROP COLUMN geom;
ALTER TABLE Stops
DROP COLUMN geometry_7844;
''')

InternalError: (psycopg2.errors.InternalError_) transform: File not found or invalid (1029)

[SQL: 
ALTER TABLE Stops
ADD COLUMN geom GEOMETRY(Point, 4326);
UPDATE Stops
SET geom = ST_SetSRID(ST_MakePoint(longitude, latitude), 4326);
ALTER TABLE Stops
ADD COLUMN geometry_7844 GEOMETRY(Point, 7844);
UPDATE Stops
SET geometry_7844 = ST_Transform(geom, 7844);

UPDATE Stops s
SET sa2_code = sa2.sa2_code
FROM SA2 sa2
WHERE ST_Within(s.geometry_7844, sa2.geometry);

DELETE FROM Stops
WHERE sa2_code IS NULL;

ALTER TABLE Stops
DROP COLUMN geom;
ALTER TABLE Stops
DROP COLUMN geometry_7844;
]
(Background on this error at: https://sqlalche.me/e/14/2j85)

In [25]:
db.execute('''
ALTER TABLE Polls
ADD COLUMN geom GEOMETRY(Point, 4326);
UPDATE Polls
SET geom = ST_SetSRID(ST_MakePoint(longitude, latitude), 4326);
ALTER TABLE Polls
ADD COLUMN geometry_7844 GEOMETRY(Point, 7844);
UPDATE Polls
SET geometry_7844 = ST_Transform(geom, 7844);

UPDATE Polls p
SET sa2_code = sa2.sa2_code
FROM SA2 sa2
WHERE ST_Within(p.geometry_7844, sa2.geometry);

DELETE FROM Polls
WHERE sa2_code IS NULL;

ALTER TABLE Polls
DROP COLUMN geom;
ALTER TABLE Polls
DROP COLUMN geometry_7844;
''')

In [26]:
db.execute('''
UPDATE School s
SET SA2_code = sa2.SA2_code
FROM SA2 sa2
WHERE ST_Within(s.geometry, sa2.geometry);

DELETE FROM School
WHERE sa2_code IS NULL;
''')

# Calculate the Zbusiness

Extract data from database which industries are "Accommodation and Food Services" and "Information Media and Telecommunications". Then calculate the total number of businesses in these industries in each SA2 region. What's more, How busy Accommodation and Food Services are is a direct indicator of how attractive and active an area is, The active degree of Information Media and Telecommunications can reflect the degree of regional technology and information mobility, and is an indicator of the vitality of modern cities.

After that, the z-score formula is given by $z = \frac{X - \mu}{\sigma}$, where $X$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

The window function OVER() is used here to calculate the mean and standard deviation of all selected data

In [27]:
Zbusiness_data = db.execute('''
WITH selected AS(
    SELECT
        sa2_code,
        sa2_name,
        SUM(total_businesses) AS selected_industries
    FROM
        businesses
    WHERE industry_name IN ('Accommodation and Food Services', 'Information Media and Telecommunications')
    GROUP BY 
        sa2_code, 
        sa2_name
)

SELECT
    sa2_code,
    sa2_name,
    (selected_industries - AVG(selected_industries) OVER()) / STDDEV(selected_industries) OVER() AS zbusiness
FROM selected;

''')


for row in Zbusiness_data:
    print(row)
    Zbusiness = row[2]
    print(f"Zbusiness: {Zbusiness}")

('101021007', 'Braidwood', Decimal('-0.41207323345593395337'))
Zbusiness: -0.41207323345593395337
('101021008', 'Karabar', Decimal('-0.60081355394015007548'))
Zbusiness: -0.60081355394015007548
('101021009', 'Queanbeyan', Decimal('-0.01362144576703325115'))
Zbusiness: -0.01362144576703325115
('101021010', 'Queanbeyan - East', Decimal('-0.50644339369804201442'))
Zbusiness: -0.50644339369804201442
('101021012', 'Queanbeyan West - Jerrabomberra', Decimal('-0.57984240721968161746'))
Zbusiness: -0.57984240721968161746
('101021610', 'Googong', Decimal('-0.61129912730038430448'))
Zbusiness: -0.61129912730038430448
('101021611', 'Queanbeyan Surrounds', Decimal('-0.09750603264890708319'))
Zbusiness: -0.09750603264890708319
('101031013', 'Bombala', Decimal('-0.62178470066061853349'))
Zbusiness: -0.62178470066061853349
('101031014', 'Cooma', Decimal('-0.10799160600914131220'))
Zbusiness: -0.10799160600914131220
('101031015', 'Cooma Surrounds', Decimal('-0.46450110025710509840'))
Zbusiness: -0.464

# Calculate the Zstops

Zstops used to quantify the standardised deviation of the number of public transport stops in each SA2 region relative to the average number of stops throughout the Sydney region

the z-score formula is given by $z = \frac{X - \mu}{\sigma}$, where $X$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

The window function OVER() is used here to calculate the mean and standard deviation of all selected data

In [43]:
Zstops_data = db.execute('''
WITH stop_counts AS (
    SELECT 
        sa2_code, 
        COUNT(stop_id) AS total_stops
    FROM stops
    GROUP BY sa2_code
)

SELECT 
    sa2_code, 
    (total_stops - AVG(total_stops) OVER()) / (STDDEV(total_stops) OVER()) AS zstops
FROM stop_counts;

''')

for row in Zstops_data:
    print(row)
    Zstops = row[1]
    print(f"Zstops: {Zstops}")

(None, None)
Zstops: None


# Calculate the Zpolls

Zpolls assess the standard deviation of the number of polling stations in each SA2 region from the average of the entire dataset.

the z-score formula is given by $z = \frac{X - \mu}{\sigma}$, where $X$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

The window function OVER() is used here to calculate the mean and standard deviation of all selected data

In [33]:
Zpolls_data = db.execute('''
WITH poll_counts AS (
    SELECT
        sa2_code,
        COUNT(polling_place_id) AS total_polls
    FROM 
        polls
    GROUP BY 
        sa2_code
)

SELECT
    sa2_code,
    (total_polls - AVG(total_polls) OVER()) / (STDDEV(total_polls) OVER()) AS zpolls
FROM poll_counts;
''')

for row in Zpolls_data:
    print(row)
    Zpolls = row[1]
    print(f"Zpolls: {Zpolls}")

('128011606', Decimal('-0.48426619268918308816'))
Zpolls: -0.48426619268918308816
('125041588', Decimal('0.39621779401842252667'))
Zpolls: 0.39621779401842252667
('127011504', Decimal('-0.04402419933538028074'))
Zpolls: -0.04402419933538028074
('116021632', Decimal('-0.92450818604298589557'))
Zpolls: -0.92450818604298589557
('112031250', Decimal('0.39621779401842252667'))
Zpolls: 0.39621779401842252667
('102021056', Decimal('0.83645978737222533409'))
Zpolls: 0.83645978737222533409
('103041076', Decimal('1.27670178072602814150'))
Zpolls: 1.27670178072602814150
('126021500', Decimal('-0.48426619268918308816'))
Zpolls: -0.48426619268918308816
('115041301', Decimal('0.39621779401842252667'))
Zpolls: 0.39621779401842252667
('101051540', Decimal('2.5974277607874366'))
Zpolls: 2.5974277607874366
('122031693', Decimal('-0.48426619268918308816'))
Zpolls: -0.48426619268918308816
('128011605', Decimal('-1.3647501793967887'))
Zpolls: -1.3647501793967887
('116021562', Decimal('-1.3647501793967887')

# Calculate the Zschools

Zschools uses a standardised approach to measure where the number of school catchments in each region compares to the average for the entire Sydney region. This can help identify which regions are relatively rich in educational resources and which may need more attention and resource input.

the z-score formula is given by $z = \frac{X - \mu}{\sigma}$, where $X$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

The window function OVER() is used here to calculate the mean and standard deviation of all selected data

In [38]:
Zschool_data = db.execute('''
WITH school_counts AS (
    SELECT 
        sa2_code, 
        COUNT(use_id) AS total_schools
    FROM 
        school
    GROUP BY 
        sa2_code
)


SELECT 
    sa2_code, 
    (total_schools - AVG(total_schools) OVER()) / (STDDEV(total_schools) OVER()) AS zschools
FROM school_counts;

''')

for row in Zschool_data:
    print(row)
    Zschool = row[1]
    print(f"Zschool: {Zschool}")

('101051539', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('101051540', Decimal('0.47490195177999327675'))
Zschool: 0.47490195177999327675
('102011030', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('103021062', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('103021069', Decimal('0.47490195177999327675'))
Zschool: 0.47490195177999327675
('103031071', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('103041076', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('103041079', Decimal('0.47490195177999327675'))
Zschool: 0.47490195177999327675
('104011081', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('104011082', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('104021087', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('104021089', Decimal('-0.44696654285175829136'))
Zschool: -0.44696654285175829136
('10501109